# House Prices - Advanced Regression Techniques

Author: Renaud Gaudron\
Latest update: 12/10/2021

### Importing the relevant python libraries

In [12]:
# General statistical/maths libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold = 1e6)
from scipy import stats

# Data visualisation libraries
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Machine Learning libraries
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
import optuna
from sklearn import preprocessing

### Importing the data & cleanup

In [13]:
data_train=pd.read_csv('train.csv') # Training set
data_test=pd.read_csv('test.csv') # Testing set

data_train.head() # Checking the first 10 rows of the training set

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [14]:
def cleanup(dataframe):
    col_list=list(dataframe.columns.values) # Listing all columns
        
    # Replacing grades by numbers    
    replace_grade_list=[
        'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC'
    ]
    
    for element in replace_grade_list:
        dataframe[element].replace(['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], [5, 4, 3, 2, 1, 0], inplace=True)       
     
    # Replacing strings by numbers for the relevant features
    dataframe['Functional'].replace(['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'], range(8,0,-1), inplace=True)  
    dataframe['GarageFinish'].replace(['Fin', 'RFn', 'Unf', 'NA'], range(4,0,-1), inplace=True)  
    dataframe['BsmtExposure'].replace(['Gd', 'Av', 'Mn', 'No', 'NA'], range(5,0,-1), inplace=True)  
    dataframe['LandSlope'].replace(['Gtl', 'Mod', 'Sev'], range(3,0,-1), inplace=True)  
    dataframe['PavedDrive'].replace(['Y', 'P', 'N'], range(3,0,-1), inplace=True)  
    dataframe['CentralAir'].replace(['N', 'Y'], [0, 1], inplace=True) 
    dataframe['Street'].replace(['Grvl', 'Pave'], [0, 1], inplace=True) 
    dataframe['Fence'].replace(['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'], range(5,0,-1), inplace=True)  
    dataframe['MiscFeature'].replace(['Shed'], 1, inplace=True)  
    dataframe['MiscFeature'].where(dataframe['MiscFeature']==1, other=0, inplace=True)  
    
    # One-hot encoding for all relevant features        
    one_hot_list=[
        'MSZoning', 'LotShape', 'LandContour', 'LotConfig', 'Utilities', 'BldgType',
        'HouseStyle', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'SaleType', 'SaleCondition',
        'Alley', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'BsmtFinType1', 'BsmtFinType2', 'Condition1', 'Condition2'
    ]
    
    for element in one_hot_list:     
        dataframe = dataframe.join(pd.get_dummies(dataframe[element], prefix=element))
        dataframe.drop(columns=element, inplace=True)
  
    # Computing the log of the sale price and the sale price per area
    if 'SalePrice' in list(dataframe.columns.values):
        dataframe['SalePriceLog']=dataframe['SalePrice'].apply(np.log)      
        dataframe['SalePricePerArea']=dataframe['SalePrice']/dataframe['GrLivArea']
    
    # Setting the index
    dataframe.set_index('Id', inplace=True)        
    
    # Dealing with NaN values using the most frequent occurrence
    NaN_list_freq=[
        'BsmtQual', 'BsmtCond', 'BsmtFullBath',
        'BsmtHalfBath', 'GarageCars', 'Fence', 'PoolQC', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
        'BsmtExposure', 'KitchenQual', 'Functional'
    ]
    
    for element in NaN_list_freq:
        dataframe[element] = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit(dataframe[[element]]).transform(dataframe[[element]]).ravel()

    # Dealing with NaN values using the mean
    NaN_list_mean=['MasVnrArea','LotFrontage', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'GarageArea', 'TotalBsmtSF', 'GarageYrBlt']
    
    for element in NaN_list_mean:
        dataframe[element] = SimpleImputer(missing_values=np.nan, strategy='mean').fit(dataframe[[element]]).transform(dataframe[[element]]).ravel()
      
    return dataframe

In [15]:
# Initial cleanup (see function above)
data_train=cleanup(data_train)
data_test=cleanup(data_test)

# Mean encoding of 'Neighborhood' using the sale price per area
data_neigh=data_train.groupby(['Neighborhood']).mean()
list_neigh=data_train.Neighborhood.unique()

for el in list_neigh:
    data_train['Neighborhood'].replace(el, data_neigh['SalePricePerArea'].loc[el], inplace=True) 
    data_test['Neighborhood'].replace(el, data_neigh['SalePricePerArea'].loc[el], inplace=True)
    
# Mean encoding of 'MSSubClass' using the sale price per area    
data_class=data_train.groupby(['MSSubClass']).mean()
list_class=data_train.MSSubClass.unique()

for el2 in list_class:    
    data_train['MSSubClass'].replace(el2, data_class['SalePricePerArea'].loc[el2], inplace=True) 
    data_test['MSSubClass'].replace(el2, data_class['SalePricePerArea'].loc[el2], inplace=True) 

# Dropping 'SalePricePerArea'    
data_train.drop(columns='SalePricePerArea', inplace=True)

# Changing the data type to float for all features    
data_train=data_train.astype('float64')
data_test=data_test.astype('float64')

In [17]:
data_train.head()

,MSSubClass,LotFrontage,LotArea,Street,LandSlope,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Utilities_AllPub,Utilities_NoSeWa,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Alley_Grvl,Alley_Pave,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,SalePriceLog
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,118.371161,65.0,8450.0,1.0,3.0,136.783680,7.0,5.0,2003.0,2003.0,196.0,4.0,3.0,4.0,3.0,2.0,706.0,0.0,150.0,856.0,5.0,1.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,8.0,8.0,0.0,4.0,2003.0,3.0,2.0,548.0,3.0,3.0,3.0,0.0,61.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,2.0,2008.0,208500.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.

In [18]:
# Checking if there is any NaN value left
data_train_NaN=data_train[data_train.isnull().any(axis=1)]
data_train_NaN.head()

,MSSubClass,LotFrontage,LotArea,Street,LandSlope,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Utilities_AllPub,Utilities_NoSeWa,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Alley_Grvl,Alley_Pave,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,SalePriceLog
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


### Feature Selection

In [8]:
# Selecting the best features using the Pearson correlation coefficient and associated p-value

Features=list(set(data_train.columns.values)-{'SalePrice', 'SalePriceLog', 'Id'})
Best_Feat=[]

for feature in sorted(Features):
    try:
        pearson_coef, p_value = stats.pearsonr(data_train['SalePrice'], data_train[feature])
        if abs(pearson_coef)>0.01 and p_value<0.05: # Selecting the features that are statistically significant
            print(feature,"- Sale Price: the Pearson correlation coefficient is", round(pearson_coef,3), " with a P-value of", format(p_value,".3E")) 
            Best_Feat.append(feature)
    except:
        pass


1stFlrSF - Sale Price: the Pearson correlation coefficient is 0.606  with a P-value of 5.395E-147
2ndFlrSF - Sale Price: the Pearson correlation coefficient is 0.319  with a P-value of 5.764E-36
Alley_Grvl - Sale Price: the Pearson correlation coefficient is -0.139  with a P-value of 9.289E-08
BedroomAbvGr - Sale Price: the Pearson correlation coefficient is 0.168  with a P-value of 9.927E-11
BldgType_1Fam - Sale Price: the Pearson correlation coefficient is 0.137  with a P-value of 1.336E-07
BldgType_2fmCon - Sale Price: the Pearson correlation coefficient is -0.097  with a P-value of 1.951E-04
BldgType_Duplex - Sale Price: the Pearson correlation coefficient is -0.115  with a P-value of 1.125E-05
BldgType_Twnhs - Sale Price: the Pearson correlation coefficient is -0.099  with a P-value of 1.578E-04
BsmtCond - Sale Price: the Pearson correlation coefficient is 0.159  with a P-value of 8.988E-10
BsmtExposure - Sale Price: the Pearson correlation coefficient is 0.362  with a P-value of 

In [9]:
X_norm=data_train[Best_Feat].append(data_test[Best_Feat]) # Appending the training and testing sets

# Normalising the features
X_norm=preprocessing.StandardScaler(with_mean=True, with_std=True).fit_transform(X_norm)

# Defining the feature matrices and output vector
Y=data_train['SalePriceLog'].values
X=X_norm[0:len(Y),:]  
X_test=X_norm[len(Y):len(X_norm),:]

# Checking the shape of the resulting matrices
print('Shape of Y:', Y.shape)
print('Shape of X:', X.shape)
print('Shape of X_test:',X_test.shape)

Shape of Y: (1460,)
Shape of X: (1460, 119)
Shape of X_test: (1459, 119)


### Training the ML algorithm

In [19]:
# Building the Machine Learning algorithm
# Algorithm: Gradient Boosting Regressor
# Hyperparameter optimisation with Optuna

clf = GradientBoostingRegressor(loss='ls', learning_rate=0.05, random_state=2)

# Parameters to be explored
param_distributions = { 
    "ccp_alpha": optuna.distributions.UniformDistribution(0, 1e-5),
    "subsample": optuna.distributions.UniformDistribution(0.5, 1),
    "n_estimators": optuna.distributions.IntUniformDistribution(1, 1000),  
    "max_depth": optuna.distributions.IntUniformDistribution(1, 5),
    "learning_rate": optuna.distributions.LogUniformDistribution(5e-4, 1.0)
}

# Finding the optimal hyperparameters using OptunaSearchCV
optuna_search = optuna.integration.OptunaSearchCV(
    clf, param_distributions, n_trials=500, verbose=2, n_jobs=-1, cv=5, scoring='neg_root_mean_squared_error'
)

optuna_search.fit(X, Y)

print("Best trial:")
trial = optuna_search.study_.best_trial

print("  Value: ", -trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    
# Checking the accuracy on the training set using the root-mean squared error    
pred_GB=optuna_search.predict(X)
acc_training_GB=metrics.mean_squared_error(Y,pred_GB, squared=False)

print('\nLogarithmic RMSE - Training set:',acc_training_GB)
print('Mean Logarithmic RMSE - CV set:',-trial.value)

# Predicting the prices of the houses in the testing set
Y_test = np.exp(optuna_search.predict(X_test))

<ipython-input-19-d63ce8692cc4>:17: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
C:\Users\Renaud\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass classifier=False as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[I 2021-10-12 11:09:27,186] A new study created in memory with name: no-name-10909a69-4d14-43e2-98e2-7f532e0cd416
[I 2021-10-12 11:09:27,187] Searching the best hyperparameters using 1460 samples...
C:\Users\Renaud\anaconda3\lib\site-packages\optuna\study\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2021-10-12 11:09:29,566] Trial 6 finished with value: -0.173480664269

Best trial:
  Value:  0.11625409549134584
  Params: 
    ccp_alpha: 2.2006290998468203e-06
    subsample: 0.5397381721655918
    n_estimators: 998
    max_depth: 3
    learning_rate: 0.03098462564714559

Logarithmic RMSE - Training set: 0.051160674643017504
Mean Logarithmic RMSE - CV set: 0.11625409549134584


### Writing the results to a file

In [20]:
final_data=np.column_stack((data_test.index.astype(int), np.transpose(Y_test).astype(float)))
np.savetxt("Prediction.csv", final_data, delimiter=",", header='Id,SalePrice', fmt='%d,%f', comments='')